In [2]:
import pandas as pd
import numpy as np
import sys
import hdbscan

In [3]:
# Load up code to run ICIM
# Available from : https://github.com/felixhorns/FlyPN
sys.path.append("../code/ICIM")
import sct
reload(sct)
# Used because of nature of ICIM library
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
dc_genes = pd.read_csv("../data/07_dropclust_data/dropclust_pca_genes.csv", header=None)
dc_genes.columns = ['symbol']
# Remove Hsp's

In [5]:
dc_genes = dc_genes[~dc_genes.symbol.str.contains('Hsp')]

In [6]:
dc_genes = list(dc_genes.symbol)

In [7]:
# Set up to pull out the list of genes in the pruned expression table
df = pd.read_csv("../data/02_filtered_kenyon_cells/CLEAN_LOG2TRANSFORM_kenyon_cells.csv", index_col='symbol')

In [8]:
df.shape

(17473, 2848)

In [9]:
# ended up with a random row nan whose entire row is 0 counts - FAKE GENE
print('Shape of df with nan index : ' + str(df[df.index.isna()].shape))
print('total sum of expression values : ' + str(sum(df[df.index.isna()].iloc[0])))
print('drop row')
df = df[~df.index.isna()]

Shape of df with nan index : (1, 2848)
total sum of expression values : 0.0
drop row


In [10]:
df.shape

(17472, 2848)

In [11]:
# Generate a filtered df based on these genes
f_df = df.copy(deep=True)
f_df = f_df.loc[dc_genes]

In [12]:
f_df.head()

,ACATACGAGGGCTTCC-DGRP-551_0d_r1,ACCCACTTCACTCTTA-DGRP-551_0d_r1,ACCGTAAAGATAGTCA-DGRP-551_0d_r1,ACTTACTAGTGGTAAT-DGRP-551_0d_r1,ACTTGTTCATGGTTGT-DGRP-551_0d_r1,ACTTTCATCAATAAGG-DGRP-551_0d_r1,AGATCTGCAACAACCT-DGRP-551_0d_r1,AGATCTGTCTACTCAT-DGRP-551_0d_r1,AGCGGTCCATTTCACT-DGRP-551_0d_r1,AGCTCTCGTTTGACTG-DGRP-551_0d_r1,...,GGCAATTCATGGATGG-w1118_15d_r1,GTCCTCAGTTGCGCAC-w1118_15d_r1,GTGCAGCGTACCGTAT-w1118_15d_r1,TATGCCCTCTATGTGG-w1118_15d_r1,TCATTACAGAGGTTGC-w1118_15d_r1,TCGAGGCAGCTATGCT-w1118_15d_r1,TTCTACATCAGTGTTG-w1118_15d_r1,AAGTCTGTCTGGTTCC-w1118_30d_r1,ACATACGTCGGATGGA-w1118_30d_r1,GAAATGACAAGACACG-w1118_30d_r1
symbol,,,,,,,,,,,,,,,,,,,,,
mamo,2.000000,4.392317,0.0,3.906891,2.321928,4.087463,4.523562,4.857981,1.000000,3.321928,...,1.584963,3.700440,0.000000,2.321928,2.807355,2.321928,3.906891,4.700440,2.321928,5.247928
pros,1.584963,3.906891,5.0,3.584963,1.584963,2.807355,3.700440,1.584963,0.000000,2.584963,...,1.000000,2.807355,4.392317,2.584963,3.321928,2.321928,2.000000,2.807355,2.000000,3.169925
CG32264,1.000000,1.000000,2.0,1.000000,0.000000,1.584963,2.000000,2.584963,2.000000,1.000000,...,1.584963,0.000000,2.000000,1.000000,0.000000,0.000000,1.000000,2.321928,1.000000,2.000000
alphagamma-element:CR32865,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.584963,0.000000,0.000000,0.000000
Rbp6,3.321928,3.700440,0.0,2.807355,3.459432,2.321928,2.807355,4.169925,2.807355,2.000000,...,2.000000,0.000000,0.000000,3.459432,0.000000,1.584963,2.807355,2.321928,1.000000,2.584963


In [13]:
f_df.shape

(193, 2848)

In [14]:
# Check to make sure each gene is present in at least 3 cells
def check_min_num_cells(row):
    num_pos = 0
    for gene_count in row:
        if gene_count > 0:
            num_pos += 1
            if num_pos >= 3:
                return True
    return False
   

min_num_cells = f_df.apply(check_min_num_cells, axis=1)
f_df = f_df[min_num_cells]

In [15]:
f_df.shape

(193, 2848)

In [16]:
# load up metadata that corresponds to full df
metadata = pd.read_csv("../data/02_filtered_kenyon_cells/metadata_kenyon_cells.csv")
metadata = metadata.set_index("CellID")

In [17]:
# Calculate TSNE
reload(sct)
from sklearn.manifold import TSNE
myTSNE = sct.TSNE(f_df, df, metadata)
myTSNE.calc_TSNE(perplexity=30, learning_rate=250, early_exaggeration=4.0, method="exact", random_state=1)

[t-SNE] Computed conditional probabilities for sample 1000 / 2848
[t-SNE] Computed conditional probabilities for sample 2000 / 2848
[t-SNE] Computed conditional probabilities for sample 2848 / 2848
[t-SNE] Mean sigma: 0.109567
[t-SNE] Iteration 50: error = 18.0508210, gradient norm = 0.0074234 (50 iterations in 14.997s)
[t-SNE] Iteration 100: error = 17.6930247, gradient norm = 0.0067211 (50 iterations in 14.788s)
[t-SNE] Iteration 150: error = 17.6057971, gradient norm = 0.0083871 (50 iterations in 15.655s)
[t-SNE] Iteration 200: error = 17.5672345, gradient norm = 0.0068668 (50 iterations in 15.170s)
[t-SNE] Iteration 250: error = 17.5437970, gradient norm = 0.0050587 (50 iterations in 15.375s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 17.543797
[t-SNE] Iteration 300: error = 2.1266124, gradient norm = 0.0007879 (50 iterations in 15.265s)
[t-SNE] Iteration 350: error = 1.9899136, gradient norm = 0.0003427 (50 iterations in 15.568s)
[t-SNE] Iteration 400: err

In [18]:
# Using HDBSCAN to call clusters automatically, generate metadata for them
clusterer = hdbscan.HDBSCAN(min_cluster_size=30, min_samples=3).fit(myTSNE.X_tsne)
labels_HDBSCAN = clusterer.labels_

In [19]:
print "Clusters", max(labels_HDBSCAN)+1

Clusters 4


In [20]:
zero_axis = myTSNE.X_tsne[:,0]
one_axis = myTSNE.X_tsne[:,1]
myTSNE.df_libs['x'] = zero_axis
myTSNE.df_libs['y'] = one_axis
myTSNE.df_libs['hdb_clust'] = labels_HDBSCAN

In [21]:
def gen_kc_type(row):
    if int(row.cell_type_id) == 8:
        return 'G-KC'
    elif int(row.cell_type_id) == 22:
        return 'a/b-KC'
    elif int(row.cell_type_id) == 28:
        return "a'/b'-KC"
    
myTSNE.df_libs['subtype'] = myTSNE.df_libs.apply(gen_kc_type, axis=1)

In [32]:
icim_data = pd.read_csv("../data/03_ICIM_analysis/KC_ICIM_TSNE_data.csv")
to_add = icim_data.set_index('CellID')['hdb_clust'].to_frame(name='icim_clust')

In [34]:
myTSNE.df_libs = myTSNE.df_libs.join(to_add)

In [35]:
myTSNE.df_libs.to_csv("../data/07_dropclust_data/dc_gene_TSNE.csv")